In [4]:
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

In [2]:
loaded_faiss_index = faiss.read_index("my_faiss_index")
query_text = "what's happening with trump"

In [5]:
def encode_texts(texts, model, tokenizer):
    # Tokenize and encode the texts using the provided model and tokenizer
    encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    
    # Forward pass to get embeddings
    with torch.no_grad():
        model_output = model(**encoded_texts)
    
    # Return the embeddings
    return model_output.last_hidden_state.mean(dim=1).numpy()

def create_faiss_index(text_vectors):
    # Convert text vectors to numpy array
    text_vectors_np = np.array(text_vectors).astype('float32')

    # Instantiate an index
    index = faiss.IndexFlatL2(text_vectors_np.shape[1])

    # Train the index
    index.add(text_vectors_np)

    return index

# Example usage:
# Replace 'distilbert-base-nli-stsb-mean-tokens' with the model of your choice
model_name = 'google/bert_uncased_L-2_H-128_A-2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at google/bert_uncased_L-2_H-128_A-2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Replace 'your_query_text' with the actual query text
query_text = "what's happening with trump"

# Encode the query text using the same model and tokenizer
query_vector = encode_texts([query_text], model, tokenizer)

# Perform a similarity search
k = 3  # Number of nearest neighbors to retrieve
distances, indices = loaded_faiss_index.search(query_vector, k)

# Print the results
print("Nearest neighbors:")
for i, index in enumerate(indices[0]):
    print(f"Rank {i+1}: Index {index}, Distance: {distances[0][i]}")

Nearest neighbors:
Rank 1: Index 0, Distance: 52.65081787109375
Rank 2: Index 2, Distance: 54.03995132446289
Rank 3: Index 1, Distance: 56.208438873291016


In [7]:
final_news_array = []
for i in range(k):
    final_news_array.append(your_texts[indices[0][i]])

NameError: name 'your_texts' is not defined

In [ ]:
np.save("my_array.npy", final_news_array)